In [0]:
import subprocess
import cv2
import keras
import numpy as np 
from google.colab import drive
#import plotConfusionMatrix
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

In [0]:
#to mounth google drive as google as google drive has data
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [0]:
#listing folders of Google Drive
ls '/content/gdrive/My Drive/Leaf_data'

early_blight/  healthy/  Late_blight/


In [0]:
#obj = plotConfusionMatrix.confusionMatrix()
folders = subprocess.check_output("ls '/content/gdrive/My Drive/Leaf_data'", shell=True)
folders = folders.decode().split('\n')
folders.pop()

''

In [0]:
folders

['early_blight', 'healthy', 'Late_blight']

In [0]:
X = []
Y = []

i = 0
for folder in folders:
    images = subprocess.check_output("ls '/content/gdrive/My Drive/Leaf_data/'" + str(folder), shell=True)
    images = images.decode().split('\n')
    images.pop()
    for image in images:
        print('/content/gdrive/My Drive/Leaf_data/' + str(folder) + '/' + str(image))
        img = cv2.imread('/content/gdrive/My Drive/Leaf_data/' + str(folder) + '/' + str(image))
        img = cv2.resize(img,(120,120))
        X.append(img)
        Y.append(i)
    i += 1

X = np.array(X)
Y = np.array(Y)

X = X.astype('float32')
X /= 255

Y = keras.utils.to_categorical(Y, 3)

/content/gdrive/My Drive/Leaf_data/early_blight/001187a0-57ab-4329-baff-e7246a9edeb0___RS_Early.B 8178.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/002a55fb-7a3d-4a3a-aca8-ce2d5ebc6925___RS_Early.B 8170.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/009c8c31-f22d-4ffd-8f16-189c6f06c577___RS_Early.B 7885.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/00d8f10f-5038-4e0f-bb58-0b885ddc0cc5___RS_Early.B 8722.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/0182e991-97f0-4805-a1f7-6e1b4306d518___RS_Early.B 7015.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/02578b86-b234-4ac0-9bc3-691b5610e2bf___RS_Early.B 7562.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/0267d4ca-522e-4ca0-b1a2-ce925e5b54a2___RS_Early.B 7020.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/028f9b73-142f-499a-9c7b-d7c1ed5e5506___RS_Early.B 8546.jpeg
/content/gdrive/My Drive/Leaf_data/early_blight/034959c1-f1e8-4a79-a6d5-3c1d14efa2f3___RS_Early.B 7136.jpeg
/content/gdrive/My Drive/Lea

In [0]:
datagen = ImageDataGenerator(
    featurewise_center=True,
    featurewise_std_normalization=True,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)

xtrain, xtest, ytrain, ytest = train_test_split(X, Y, test_size=0.15, stratify=Y, random_state=42)
#print(xtrain[1])
datagen.fit(xtrain)

In [0]:
model = Sequential()

model.add(Convolution2D(32, (3, 3), activation='relu',input_shape = xtrain.shape[1:]))
model.add(Convolution2D(32, (3, 3), activation='relu'))  
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Convolution2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

model.add(Convolution2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))
 
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))

model.add(Dense(3, activation='sigmoid'))


model.compile(loss='binary_crossentropy', 
              optimizer='adam', 
              metrics=['accuracy'])

early_stopping = EarlyStopping(patience=5, verbose=1)
model_checkpoint = ModelCheckpoint("'/content/gdrive/My Drive/Leaf_data/model.h1'", save_best_only=True, verbose=1)
reduce_lr = ReduceLROnPlateau(factor=0.1, patience=2, min_lr=0.00001, verbose=1)

In [0]:
model.fit(xtrain, ytrain, shuffle=True, batch_size=30, epochs=30,callbacks=[early_stopping, model_checkpoint, reduce_lr], verbose=1)

Epoch 1/30
51/51 [==============================] - 4s 76ms/step - loss: 0.8202 - acc: 0.6078
Epoch 2/30


/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:842: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:707: RuntimeWarning: Can save best model only with val_loss available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)
/usr/local/lib/python3.6/dist-packages/keras/callbacks.py:1379: RuntimeWarning: Reduce LR on plateau conditioned on metric `val_loss` which is not available. Available metrics are: loss,acc,lr
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


51/51 [==============================] - 3s 56ms/step - loss: 0.6816 - acc: 0.5556
Epoch 3/30
51/51 [==============================] - 3s 56ms/step - loss: 0.6836 - acc: 0.5556
Epoch 4/30
51/51 [==============================] - 3s 56ms/step - loss: 0.6701 - acc: 0.6667
Epoch 5/30
51/51 [==============================] - 3s 55ms/step - loss: 0.6459 - acc: 0.6667
Epoch 6/30
51/51 [==============================] - 3s 56ms/step - loss: 0.6342 - acc: 0.6667
Epoch 7/30
51/51 [==============================] - 3s 56ms/step - loss: 0.6311 - acc: 0.6667
Epoch 8/30
51/51 [==============================] - 3s 55ms/step - loss: 0.6269 - acc: 0.6667
Epoch 9/30
51/51 [==============================] - 3s 55ms/step - loss: 0.6218 - acc: 0.6667
Epoch 10/30
51/51 [==============================] - 3s 56ms/step - loss: 0.6036 - acc: 0.6667
Epoch 11/30
51/51 [==============================] - 3s 55ms/step - loss: 0.5874 - acc: 0.6928
Epoch 12/30
51/51 [==============================] - 3s 55ms/step - l

In [0]:
score = model.evaluate(xtest, ytest)
print('Test Accuracy = ' + str(score[1]))

9/9 [==============================] - 0s 32ms/step
Test Accuracy = 0.7407407164573669


In [0]:
#prediction for single image
x = []
img_1 = cv2.imread('/content/gdrive/My Drive/Leaf_data/img.jpg')
img_1 = cv2.resize(img,(120,120))
x.append(img)
x= np.array(x)
x = x.astype('float32')
x/= 255

In [0]:
result = model.predict(x)
print(np.argmax(result[0]))

0
